In [ ]:
#import nltk
#nltk.download('wordnet_ic')
from nltk.corpus import wordnet, wordnet_ic
import pandas as pd
import csv
import scipy.stats
import re

In [ ]:
#locate the file and name the variable that will refer to the file
part1_file = '/home/hector/Documents/Language_Technology/HW1/ws353.tsv'
part2_file = '/home/hector/Documents/Language_Technology/HW1/Results_PPMI'
part2_file_tsv = '/home/hector/Documents/Language_Technology/HW1/Results_PPMI.tsv'
part3_file = '/home/hector/Documents/Language_Technology/HW1/Results_Word2Vec'
part3_file_tsv = '/home/hector/Documents/Language_Technology/HW1/Results_Word2Vec.tsv'

#read the data using pandas
part1_file_read = pd.read_csv(part1_file, sep='\t')

In [ ]:
# creates a tsv file from the PPMI results file

def tsv_creator(file1, file2):
    with open(file1, 'r') as source:
        for line in source:
            line_list = []
            word_results_1 = re.findall(r"^\w+", line)[0]
            word_results_2 = re.findall(r"(?<=\t)\w+", line)[0]
            sim_results = re.findall(r"-?\d+\.\d+$", line)[0]
            line_list.append(word_results_1)
            line_list.append(word_results_2)
            line_list.append(sim_results)
            with open(file2, 'a') as sink:
                tsv_writer = csv.writer(sink, delimiter='\t')
                tsv_writer.writerow(line_list)

tsv_creator(part3_file, part3_file_tsv)

In [ ]:
#calculates Spearman correlation between human judgement and PPMI and Word2Vec results

def spearman (tsv_1, tsv_2):
    count_total_pairs = 0
    count_supported_pairs = 0
    with open(tsv_1, 'r') as tsv_file1:
        tsv_ws353 = csv.reader(tsv_file1, delimiter='\t')
        human_list = []
        results_list = []
        for row_h in tsv_ws353:
            count_total_pairs += 1
            word_h_1 = row_h[0]
            word_h_2 = row_h[1]
            human_results = row_h[2]
            human_tuple = (word_h_1, word_h_2)
            #print(human_tuple, human_results)
            with open(tsv_2, 'r') as tsv_file2:
                tsv_results = csv.reader(tsv_file2, delimiter='\t')
                for row_r in tsv_results:
                    word_results_1 = row_r[0]
                    word_results_2 = row_r[1]
                    sim_results = row_r[2]
                    results_tuple = (word_results_1, word_results_2)
                    if human_tuple != results_tuple:
                        continue
                    elif human_tuple == results_tuple:
                        count_supported_pairs += 1
                        human_list.append(human_results)
                        results_list.append(sim_results)
    (stat, p) = scipy.stats.spearmanr(results_list, human_list)
    print(f"Spearman rho:\t{stat:.4f}\t(P = {p:.4f})")
    print("Supported pairs:", count_supported_pairs, '\t', "Total pairs:", count_total_pairs)
    print('Coverage =', round((count_supported_pairs / count_total_pairs) * 100, 4), '%')

spearman(part1_file, part3_file_tsv)